<a href="https://colab.research.google.com/github/mohmirzabr/SIMP/blob/main/1.%20Simple%20Mathematical%20Benchmarking%20for%20Natural%20Language%20Processing%20Models/MAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title If something went wrong with ollama. Turn this cell on!
import subprocess
process = subprocess.Popen ("ollama serve", shell=True)

In [ ]:
# 🎨 Math LLM Study Assistant Experiment Notebook
#@title 👨‍🔬 Simple Mathematical Benchmarking for Natural Language Processing Models
#@markdown This notebook is intended for research purposes only. Based on a legacy notebook from the repository which is developed together with ChatGPT by OpenAI that I attributed to. This notebook guides you step-by-step to compare atleast three natural language processing models from Ollama on basic math questions from GSK8M (openai/gsk8m).
#@markdown <br> <br> This project is so simple that it only could benchmark simple mathemathics like GSM8K. As the result comparing only compare the numbers from the generation result and looking for the supposedly correct number, whether it is mentioned in the generation result or not.
#@markdown <br> <br> In conclusion, this notebook isn't suitable for advanced benchmarking and only can process simple mathematical prompts.
#@markdown <br> <br> Visit me on [GitHub](https://github.com/mohmirzabr)

In [ ]:
#@title 0. ⚠️ MAKE SURE YOU'RE USING NVIDIA T4 AS THE GPU AND PYTHON 3 FOR THE RUNTIME TYPE
#@markdown This cell will make sure whether you're having your NVIDIA T4 enabled or not.
!nvidia-smi

In [ ]:
#@title 1. 🧰 Install Ollama
# @markdown In this cell, we will install the backend first. In which, we'll be using Ollama.

# 🚀 Installing Ollama
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
#@title 2. 🧰 Prepare the Model
# @markdown In this cell, we'll also pull the model to be tested in the next cells.
# @markdown  <br> <br> Below are the variables of each model that you want to use. Make sure that the models are existent in Ollama to be downloaded.
model = "tinyllama:1.1b" # @param ["tinyllama:1.1b","phi3:3.8b","mistral:7b"] {"allow-input":true}

# 🔥 Enabling Ollama
import subprocess
process = subprocess.Popen ("ollama serve", shell=True)

# ♻️ Pulling the models.
!ollama pull {model}

# 📦 Confirming the models that are ready.
!ollama list

In [ ]:
#@title 2. 📂 Load Math Questions
#@markdown This cell is where we're going to put the samples to be tested to each model. This cell will create a file named 'math_questions.txt' with lines: question<TAB>answer (except if the file existed in the first place).
#@markdown <br> <br> If you haven't prepare any datasets, you can use the template below.
templatequestions = False # @param {"type":"boolean"}

#@markdown <br> <br> Below are the variables that are going to be used for scraping the dataset from Hugging Face to 'math_questions.txt'.
#@markdown <br> <br> 1. 🧰 Which dataset you're going to refer to?
dataset = "openai/gsm8k" # @param {"type":"string"}
branch = "main" # @param {"type":"string"}
#@markdown <br> <br> 2. 📊 How many examples to use?
NUM_SAMPLES = 30 # @param {"type":"number"}

if templatequestions == False:
  from pathlib import Path

  questions_file = Path('math_questions.txt')

  if not questions_file.exists():
      # 🔧 1. Install and import the HF datasets library
      !pip install -U datasets
      from datasets import load_dataset

      # 🔍 2. Load the GSM8K training split
      gsm8k = load_dataset(dataset, branch, split="train")

      # ✂️ 3. Extract question + final answer (after "####")
      samples = []
      for ex in gsm8k.select(range(NUM_SAMPLES)):
          q_text = ex["question"].strip()
          # the answer field has steps + "#### <final>"
          raw_ans = ex["answer"]
          # split on "####" to get the last line as our truth
          truth = raw_ans.split("####")[-1].strip()
          samples.append((q_text, truth))

      # 📝 4. Save those to math_questions.txt as "question␉answer"
      with open(questions_file, 'w') as f:
          for q, a in samples:
              f.write(f"{q}\t{a}\n")

      print(f"✅ Pulled {NUM_SAMPLES} samples from {dataset} and wrote '{questions_file}'")

  # 📖 5. Now read back the file into `questions` for your experiment loop
  questions = []
  with open(questions_file) as f:
      for line in f:
          q, a = line.strip().split("\t")
          questions.append((q, a))

  print(f"✅ Loaded {len(questions)} questions for testing!")
elif templatequestions == True:
  from pathlib import Path

  questions_file = Path('math_questions.txt')
  if not questions_file.exists():
      # Create a sample file with simple math problems
      sample = [
          "12 x 7\t84",
          "17 * 24\t408",
          "256 / 8\t32",
          "13 + 49\t62"
      ]
      with open(questions_file, 'w') as f:
          for line in sample:
              f.write(line + '\n')  # 📝 writing sample questions

  # Read the questions into a list
  questions = []
  with open(questions_file) as f:
      for line in f:
          q, a = line.strip().split("\t")
          questions.append((q, a))

  print(f"✅ Loaded {len(questions)} questions for testing!")

In [ ]:
#@title 🏁 3. Set Up Results Logging
#@markdown We will record: question, model, answer, correct (True/False), time in seconds and put it into a CSV file to be processed by the next cell and then compared for conclusion.

import csv

output_file = f"results_{model.replace(':','_')}.csv"
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Header row for our CSV
    writer.writerow(['question', 'model', 'answer', 'correct', 'time_s', 'mem_delta_MiB'])
print(f"🗄️  Results will be saved in: {output_file}")

In [ ]:
#@title 4. 🚀 Experiment is running!
#@markdown This is the cell where everything begin to run, and the GPU RAM start to shrinking. The notebook is processing all of the questions and comparing the answer at the same time!

import time, subprocess, csv, re

selected_model = model  # your dropdown variable

# Helper to read GPU memory (unchanged)
def get_gpu_mem():
    out = subprocess.run(
        ['nvidia-smi',
         '--query-gpu=memory.used',
         '--format=csv,noheader,nounits'],
        stdout=subprocess.PIPE,
        text=True
    )
    return int(out.stdout.strip())

# Sample memory before loading/running the model
mem_before = get_gpu_mem()
print(f"🏷️  Running model: {selected_model}")
print(f"▶️  GPU memory BEFORE: {mem_before} MiB\n")

# Run through all questions
for question, truth in questions:
    # 1️⃣ Print the question
    print(f"\n🔍 [Q] {question}")

    # 2️⃣ Run inference
    t0 = time.time()
    result = subprocess.run(
        ['ollama', 'run', selected_model, question],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    elapsed = time.time() - t0

    # 3️⃣ Memory after (model is already loaded)
    mem_after = get_gpu_mem()
    mem_delta = mem_after - mem_before

    # 4️⃣ Parse model output
    if result.returncode != 0:
        answer = result.stderr.strip()
    else:
        lines = [l for l in result.stdout.strip().split('\n') if l.strip()]
        answer = lines[-1] if lines else ""

    # 5️⃣ Show diagnostics
    print(f"🔎 Looking for numeric token: '{truth}'")
    found_numbers = re.findall(r'\d+\.?\d*', answer)
    print(f"🔢  Extracted numbers: {found_numbers}")

    # 6️⃣ Check correctness
    correct = truth in found_numbers
    mark = '✅' if correct else '❌'

    # 7️⃣ Print the answer and result
    print(f"🗨️  Answer: {answer}")
    print(f"{mark}  ({elapsed:.2f}s, ΔRAM={mem_delta} MiB)")

    # 8️⃣ Log to CSV
    with open(output_file, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([
            question,
            selected_model,
            answer,
            correct,
            f"{elapsed:.2f}",
            mem_delta
        ])

# Final memory report
print(f"\n✔️  Finished running {selected_model}. Final GPU RAM used: {mem_after} MiB")

In [ ]:
#@title 5. 📊 Summarize & Compare Results
#@markdown This is where everything begin to be put into a conclusion for a single model. By using pandas, this cell loads the CSV file and compute per-model metrics: Accuracy (%), Average inference time (seconds), and Memory Usage (MiB).

# ✅ Summary Code for Single-Model Runs (New Format)
import pandas as pd
import matplotlib.pyplot as plt

# Load the result file from current model
file_path = f'results_{selected_model.replace(":", "_")}.csv'
df = pd.read_csv(file_path)

# Convert types as needed
df['correct_num'] = df['correct'].astype(int)
df['time_s'] = df['time_s'].astype(float)
df['mem_delta_MiB'] = df['mem_delta_MiB'].astype(float)

# Calculate summary metrics
accuracy = df['correct_num'].mean() * 100
avg_time = df['time_s'].mean()
avg_mem = df['mem_delta_MiB'].mean()

# Create a summary DataFrame for display
summary_df = pd.DataFrame([{
    'Model': selected_model,
    'Accuracy (%)': round(accuracy, 1),
    'Avg Latency (s)': round(avg_time, 2),
    'GPU Memory Used (MiB)': round(avg_mem, 1)
}])

# 🧾 Show table
from IPython.display import display
print("📈 Overall Performance Summary:")
display(summary_df)

In [ ]:
#@title 6. 🔀 Merge All Results & Final Comparison
#@markdown This cell will combine all CSV file named after 'result_'. Simply, it will combine all result into one single file named 'results_combined_summary.csv'.
import pandas as pd
import glob
import os

# 1️⃣ Find all result CSVs in this directory
pattern = "results_*.csv"
files = glob.glob(pattern)
if not files:
    raise FileNotFoundError(f"No files matching {pattern} found!")

# 2️⃣ Load each file and extract the model name from its filename
all_dfs = []
for fn in files:
    # e.g. fn = "results_phi3_3.8b.csv"
    model_name = os.path.basename(fn).removeprefix("results_").removesuffix(".csv")
    df = pd.read_csv(fn)
    df['model'] = model_name
    all_dfs.append(df)

# 3️⃣ Concatenate into one DataFrame
combined = pd.concat(all_dfs, ignore_index=True)

# 4️⃣ Compute per-model metrics
combined['correct_num'] = combined['correct'].astype(int)
combined['time_s']      = combined['time_s'].astype(float)
combined['mem_delta_MiB'] = combined['mem_delta_MiB'].astype(float)

summary = combined.groupby('model').agg(
    Accuracy       = ('correct_num',     lambda x: x.mean() * 100),
    Avg_Latency_s  = ('time_s',          'mean'),
    Avg_Mem_Use_MiB= ('mem_delta_MiB',   'mean')
).round(2).reset_index()

# 5️⃣ Display the final comparison table
from IPython.display import display
print("🏁 Final Comparison of All Models:")
display(summary)

# 🎉 Optional: save this summary if you like
summary.to_csv("results_combined_summary.csv", index=False)
print("Saved combined summary to results_combined_summary.csv")

# 🎉 End of experiment. Review the printed logs and summary to draw conclusions!